In [ ]:
!!molecule_generation preprocess INPUT_DIR OUTPUT_DIR TRACE_DIR --generation-order=canonical
Misc TODOs
Add a motif embedding layer to the GNN => need to look into whether there are specific gnn layers for this
Dataset => validate if it is correct
Implement function for storing Data objects in the Pytorch geometric dataset
Implement all the helper function for the pytorch geometric dataset

In [ ]:
import gzip
import pickle

with gzip.open('trace_playground/metadata.pkl.gz', 'rb') as f:
     metadata= pickle.load(f)

In [ ]:
import gzip
import pickle

with gzip.open('trace_playground/train_0/train_0.pkl.gz', 'rb') as f:
     train= pickle.load(f)

# Misc TODOs

1. Add a motif embedding layer to the GNN => need to look into whether there are specific gnn layers for this
2. Dataset => validate if it is correct
3. Implement function for storing Data objects in the Pytorch geometric dataset
4. Implement all the helper function for the pytorch geometric dataset

# Dataset
The original dataset uses a function to randomly decide whether to include or exclude a particular partial graph from the batch. Instead of that, we try to simply save all Data objects a separate .pt files => at training time, we list all the file paths and then sample from the file paths uniformly 

- Downsides => will occupy more memory: potential optimisations (TODOs) would be to save as compressed file and only decompress at training time.
- Upside => less preprocessing time at training time


There will be 1 dataset object for each of the train/validation/test splits

In [ ]:
import torch
from torch_geometric.data import Data

In [ ]:
from torch_geometric.data import Dataset
import os
class MolerDataset(Dataset):
    def __init__(
        self, 
        root, 
        raw_moler_trace_dataset_parent_folder, # absolute path 
        output_pyg_trace_dataset_parent_folder, # absolute path
        split = 'train'
        transform=None, 
        pre_transform=None, 
    ):
        super().__init__(root, transform, pre_transform)
        self._transform = transform 
        self._pre_transform = pre_transform
        self._raw_moler_trace_dataset_parent_folder = raw_moler_trace_dataset_parent_folder
        self._output_pyg_trace_dataset_parent_folder = output_pyg_trace_dataset_parent_folder
        self._split = split
        

    @property
    def raw_file_names(self):
        """Raw generation trace files output from the preprocess function of the cli. These are zipped pickle
        files."""
        raw_file_paths_folder = os.path.join(self._raw_moler_trace_dataset_parent_folder, self._split)
        assert os.path.exist(raw_file_paths_folder), f'{raw_file_paths_folder} does not exist.'
        raw_generation_trace_files = [os.path.join(file)  for file in os.listdir(raw_file_paths_folder)]
        return raw_generation_trace_files

    @property
    def processed_file_names(self):
        """Processed generatio trace objects that are stored as .pt files"""
        processed_file_paths_folder = os.path.join(self._output_pyg_trace_dataset_parent_folder, self._split)
        assert os.path.exist(raw_file_paths_folder), f'{raw_file_paths_folder} does not exist.'
        processed_generation_trace_files = [os.path.join(file)  for file in os.listdir(processed_file_paths_folder)]
        return processed_generation_trace_files
    
    @property
    def processed_file_names_size(self):
        return len(self.processed_file_names)
    

    def process(self):
        """Convert raw generation traces into individual .pt files for each of the trace steps."""
        # only call process if it was not called before
        if self.processed_file_names_size > 0:
            pass
        else:
            idx = 0
            for pkl_file in self.raw_file_names:
                # TODO refactor out this into a private function
                with gzip.open(pkl_file, 'rb') as f:
                     train= pickle.load(f)
                
                # Read data from `raw_path`.
                data = Data(...)

                if self.pre_filter is not None and not self.pre_filter(data):
                    continue

                if self.pre_transform is not None:
                    data = self.pre_transform(data)

                torch.save(data, osp.join(self.processed_dir, f'data_{idx}.pt'))
                idx += 1

    def _convert_raw_trace_steps_to_pickle(self, ):
        with gzip.open(pkl_file, 'rb') as f:
             train= pickle.load(f)
    def len(self):
        return len(self.processed_file_names)

    def get(self, idx):
        data = torch.load(osp.join(self.processed_dir, f'data_{idx}.pt'))
        return data

In [ ]:
def node_types_to_indices(self, node_types):
    node_type_to_index
    """Convert list of string representations into list of integer indices."""
    return [node_type_to_index(node_type) for node_type in node_types]


for generation_step in train_sample:
    x = generation_step.partial_node_features
    node_categorical_features = generation_step.node_categorical_features # TODO: do we need this?
    
    
    
    
    
    
    
    